In [1]:
import numpy as np
import os

# data manager and analysis
import vodex as vx
import numan as nu

# Project structure: 

Provide the project folder, then the "processed" folder will be created inside. 

As you keep going with the analysis, the folder will have the following structure: 

```
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++ WILL BE DONE in this notebook +++++++++++++++
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
processed                                               +++
│   experiment.json <-----------------------------------+++ the file that contains everything about the experiment, you are creating it once and will be reusing ever after
│   experiment_dff.json <-------------------------------+++ everything about the experiment, but loads from the dff movie, not from the raw data                   ++
└───dff_movie  <----------------------------------------+++ the dff movie :)
│   │   dff_movie_0000.tif                              +++
│   │   dff_movie_0001.tif                              +++
│   │   ...                                             +++
│++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
│++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
│++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
│
└───tscore_volumes  <----------------------------------- t-score tif files
│   │   tscore_SvB.tif <-------------------------------- t-score Stimuli vs Blank
└───spots
│   └───imaris  <--------------------------------------- ATTENTION : You need to put stuff generated by imaris into this folder!!! 
│       │   └───tscore_SvB_Statistics
│       │       │     tscore_SvB_Position.csv
│       │       │     tscore_SvB_Diameter.csv
│       │       │     ...
│   └───signals  <-------------------------------------- json files with the extracted signals, also will have the group info after you added it
│       │   spots_SvB.json
│   └───reports  <---------------------------------- tiffs and pdf with the cells significant in any pairwise comparison
│       └───all_significant  <---------------------- tiffs and pdf with all significant in any way cells
│           │   └───signals  <---------------------- pdfs with signals
│           │       │     ...
│           │   └───images <------------------------ tif masks 
│           │       │     ...
│       └───groupped  <----------------------------- tiffs and pdf where the cells are groupped based on signal shape .. or anything else you want
│           │   readme.txt  <----------------------- ATTENTION : you need to describe the groups
│           │   └───signals  <---------------------- pdfs with signals
│           │       │     ...
│           │   └───images  <----------------------- tif masks 
│           │       │     ...
```
# Set project folder
Also, if the processed folder exists, it will complain ...

In [2]:
project_folder = "D:/Code/repos/numan/notebooks/data/v1.x.x/2vs3vs5/"
project = nu.Project(project_folder)
project.create("processed")

We will also set the processed directory as our working directory, this step is important , since all the paths later are relative to this folder. Verify that the output of the cell is the "processed" folder inside your project folder. 

In [3]:
project.activate("processed")
os.getcwd()

'D:\\Code\\repos\\numan\\notebooks\\data\\v1.x.x\\2vs3vs5\\processed'

# Define experiment: 

* First you need to provide the path to the folder with the **raw data**, ```data_dir```. Note that by default the code will search for all the .tif files in that folder and will treat it as raw data. While you can exclude the unwanted files later, it is recommended that three is only raw tif data in that folder. Make sure to use either ```\\``` or ```/``` as a separator, not ```\```, as it is an escape character in python.

* provide the number of **frames per volume**

* Then you need to define the **labels**.

* Then, if your experiment has a **repeating cycle**, you will need to order labels as they appear in a cycle in and provide the corresponding duration for each label. Note that the duration is in frames, as they appear in your image data, not in seconds, not in volumes.

This will initialise experiment and output the experiment information. Read it carefully! Make sure it is all as expected! Any mistake at this step will make all the future analysis wrong.

In [4]:
# files info
data_dir = "D:/Code/repos/numan/notebooks/data/2vs3vs5/raw/"

# volumes info
frames_per_volume = 52
volume_info = vx.VolumeManager.from_dir(data_dir, frames_per_volume, fgf = 0)

#annotation info
dots = vx.Labels("number",
                   ["b","d2","d3","d5"],
                   group_info = "number of dots on the screen",
                   state_info={"b":"blank, no dots",
                               "d2": "2 dots on the screen",
                               "d3": "3 dots on the screen",
                               "d5": "5 dots on the screen"})
dot_cycle = vx.Cycle(   # note how you can use the name of the stimuli as an attribute!
                        # for this reason it's important the names don't have a space or any other special characters in them
                        [
                        dots.b,dots.d3,
                        dots.b,dots.d2,
                        dots.b,dots.d5,
                        dots.b,dots.d2,
                        dots.b,dots.d3,
                        dots.b,dots.d2,
                        dots.b,dots.d5,
                        dots.b,dots.d3,
                        dots.b,dots.d5
                        ],
                        # the duration should be in frames!
                        # here it's first listed in volumes and then multiplied by frames per volume
                        np.array(
                            [7,1,
                             5,1,
                             6,1,
                             6,1,
                             9,1,
                             5,1,
                             9,1,
                             6,1,
                             9,1]
                        )*52
                    )
dot_annotation = vx.Annotation.from_cycle(volume_info.n_frames, dots, dot_cycle,
                                          info = "number of dots on the screen, semi-random cycle")
experiment = vx.Experiment.create(volume_info, [dot_annotation], verbose=True)

Image files information :

files directory: D:\Code\repos\numan\notebooks\data\2vs3vs5\raw
files [number of frames]: 
0) 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0.ome.tif [8910]
1) 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0_1.ome.tif [8909]
2) 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0_2.ome.tif [8909]
3) 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0_3.ome.tif [8909]
4) 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0_4.ome.tif [8909]
5) 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0_5.ome.tif [8909]
6) 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0_6.ome.tif [8909]
7) 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0_7.ome.tif [4092]

Total frames : 66456
Volumes start on frame : 0
Total good volumes : 1278
Frames per volume : 52
Tailing frames (not a full volume , at the end) : 0

Annotation type: number
number of dots on the screen, semi-random cycle
Total frames : 66456

18 full cycles 

**Make sure everything above is correct** and then , if it is , save it for future use: 

In [5]:
experiment.save("experiment_raw.db")

Copied 1698 of 1698 pages...


# Create dff movie

In [9]:
project.create("processed/dff_movie")

# batch size 25 takes about 30 Gb of RAM to process
batch_size = 25 # in volumes
window_size = 15 # in volumes

In [10]:
nu.Preprocess(experiment).batch_dff('dff_movie',batch_size, window_size, verbose=True )

written frames : 0 - 17, out of 1278
written frames : 18 - 28, out of 1278
written frames : 29 - 39, out of 1278
written frames : 40 - 50, out of 1278
written frames : 51 - 61, out of 1278
written frames : 62 - 72, out of 1278
written frames : 73 - 83, out of 1278
written frames : 84 - 94, out of 1278
written frames : 95 - 105, out of 1278
written frames : 106 - 116, out of 1278
written frames : 117 - 127, out of 1278
written frames : 128 - 138, out of 1278
written frames : 139 - 149, out of 1278
written frames : 150 - 160, out of 1278
written frames : 161 - 171, out of 1278
written frames : 172 - 182, out of 1278
written frames : 183 - 193, out of 1278
written frames : 194 - 204, out of 1278
written frames : 205 - 215, out of 1278
written frames : 216 - 226, out of 1278
written frames : 227 - 237, out of 1278
written frames : 238 - 248, out of 1278
written frames : 249 - 259, out of 1278
written frames : 260 - 270, out of 1278
written frames : 271 - 281, out of 1278
written frames : 2

### Now cerate an experiment that uses dff_movie as the data
Before we make a new experiment, let's close the databse connection of the existing one.

In [11]:
experiment.close()

In [12]:
# update files info
data_dir = "dff_movie"
volume_info = vx.VolumeManager.from_dir(data_dir, frames_per_volume, fgf = 0)
# create new experiment
experiment = vx.Experiment.create(volume_info, [dot_annotation], verbose=True)

Image files information :

files directory: dff_movie
files [number of frames]: 
0) dff_movie_0000.tif [936]
1) dff_movie_0001.tif [572]
2) dff_movie_0002.tif [572]
3) dff_movie_0003.tif [572]
4) dff_movie_0004.tif [572]
5) dff_movie_0005.tif [572]
6) dff_movie_0006.tif [572]
7) dff_movie_0007.tif [572]
8) dff_movie_0008.tif [572]
9) dff_movie_0009.tif [572]
10) dff_movie_0010.tif [572]
11) dff_movie_0011.tif [572]
12) dff_movie_0012.tif [572]
13) dff_movie_0013.tif [572]
14) dff_movie_0014.tif [572]
15) dff_movie_0015.tif [572]
16) dff_movie_0016.tif [572]
17) dff_movie_0017.tif [572]
18) dff_movie_0018.tif [572]
19) dff_movie_0019.tif [572]
20) dff_movie_0020.tif [572]
21) dff_movie_0021.tif [572]
22) dff_movie_0022.tif [572]
23) dff_movie_0023.tif [572]
24) dff_movie_0024.tif [572]
25) dff_movie_0025.tif [572]
26) dff_movie_0026.tif [572]
27) dff_movie_0027.tif [572]
28) dff_movie_0028.tif [572]
29) dff_movie_0029.tif [572]
30) dff_movie_0030.tif [572]
31) dff_movie_0031.tif [572]
3

**Make sure everything except the filenames looks identical to the experiment we defined at the beginning** and then , if it does , save it for future use: 

In [13]:
experiment.save("experiment_dff.db")
experiment.close()

Copied 1696 of 1696 pages...
